In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 20)

In [2]:
df = pd.read_csv('datasets/311-last-year-requests.csv')

C:\Users\steag\AppData\Local\Temp\ipykernel_408\2081399811.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/311-last-year-requests.csv')


In [3]:
# defining boundary dates
start_date = pd.to_datetime('2023-05-01')
end_date = pd.to_datetime('2023-05-30')

# transforming date columns
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])

# partitioning dataset
df = df[(df['Created Date'] >= start_date) & (df['Created Date'] <= end_date)]
df.to_csv('datasets/311-2023-05.csv', index=False)

In [2]:
df = pd.read_csv('datasets/311-2023-05.csv')

C:\Users\steag\AppData\Local\Temp\ipykernel_4204\2412690863.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/311-2023-05.csv')


In [3]:
# number of rows
print(f"DATASET ROWS: {len(df)}")

DATASET ROWS: 262183


In [4]:
# checking null values in columns
count_rows = len(df)
print("NULL VALUES:")
for i, col in enumerate(df.columns):
    count_null_values = df[col].isnull().sum()
    perc_null_values = round(count_null_values / count_rows * 100, 2)
    print(f"+ {col}: {count_null_values} ({perc_null_values}%)")

NULL VALUES:
+ Unique Key: 0 (0.0%)
+ Created Date: 0 (0.0%)
+ Closed Date: 9840 (3.75%)
+ Agency: 0 (0.0%)
+ Complaint Type: 0 (0.0%)
+ Descriptor: 7099 (2.71%)
+ Location Type: 29815 (11.37%)
+ Incident Address: 8537 (3.26%)
+ Street Name: 8544 (3.26%)
+ Status: 0 (0.0%)
+ Community Board: 0 (0.0%)
+ Borough: 0 (0.0%)
+ Open Data Channel Type: 0 (0.0%)
+ Vehicle Type: 262124 (99.98%)
+ Taxi Company Borough: 262023 (99.94%)
+ Location: 4312 (1.64%)


In [27]:
df = pd.read_csv('datasets/311-2023-05.csv')

C:\Users\steag\AppData\Local\Temp\ipykernel_19068\2412690863.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/311-2023-05.csv')


In [28]:
df.drop(columns=['Vehicle Type', 'Taxi Company Borough'], inplace=True)

In [29]:
acronyms = {'MANHATTAN': 'MN',
            'BROOKLYN': 'BK',
            'QUEENS': 'QN',
            'BRONX': 'BX',
            'STATEN ISLAND': 'SI'}

def transform_district_name(district_name):
    number, borough = district_name.split(' ', 1)
    borough_acronym = acronyms.get(borough)
    return f"{borough_acronym}{number}"

normalize_name = lambda s: ' '.join(s.split())

for col in df.columns:
    df[col] = df[col].fillna('unknown')

cols_to_normalize = ['Complaint Type', 'Descriptor', 'Location Type',
                     'Incident Address', 'Street Name', 'Status',
                     'Borough', 'Open Data Channel Type']

for col in cols_to_normalize:
    df[col] = df[col].str.lower()
    df[col] = df[col].apply(normalize_name)

df.rename(columns={'Community Board': 'Community District'}, inplace=True)
df['Community District'] = df['Community District'].apply(transform_district_name)

df[['Latitude', 'Longitude']]

Unique Key                                              57658722
Created Date                                 2023-05-20 22:38:48
Closed Date                                  2023-12-20 11:07:09
Agency                                                      DSNY
Complaint Type                                          graffiti
Descriptor                                              graffiti
Location Type                                          comercial
Incident Address                               1484 first avenue
Street Name                                         first avenue
Status                                                    closed
Community District                                          MN08
Borough                                                manhattan
Open Data Channel Type                                   unknown
Location                  (40.77124350703734, -73.9534775413937)
Name: 0, dtype: object
